#### 【 데이터프레임 재배치/재구성 】

- DataFrame의 행 인덱스, 열 인덱스, 값 속성 설정으로 형태 변경
- 엑셀의 피벗 기능과 유사
- 긴(long) 형식 데이터  < --- > 넓은(wide) 형식으로 변경
- 주의!!!  index+columns 조합은 고유해야 함!!


[1] 모듈 로딩 <hr>

In [1]:
## 모듈 로딩
import pandas as pd 

In [ ]:
## DataFrame 인스턴스 준비
df = pd.DataFrame({
    '날짜'  : ['2025-01','2025-01','2025-02','2025-02','2025-02'],
    '지점'  : ['서울','부산','서울','부산','부산'],
    '품목'  : ['A','A','A','A','B'],
    '매출'  : [100, 120, 90, 130, 70],
    '수량'  : [10,  12,  9,  13,  7]
})

## 확인
display(df)

,날짜,지점,품목,매출,수량
0,2025-01,서울,A,100,10
1,2025-01,부산,A,120,12
2,2025-02,서울,A,90,9
3,2025-02,부산,A,130,13
4,2025-02,부산,B,70,7


[2] pivot() — 형태만 바꾸기(집계 없음) <hr>

In [ ]:
## =====================================================
## [재구성] 날짜-지점 표로, 값은 매출
## =====================================================
## 같은 (날짜, 지점) 조합이 여러 행으로 중복되면 
## ValueError: Index contains duplicate entries...
## wide = df.pivot(index='날짜', columns='지점', values='매출')

## display(wide)

In [ ]:
## =====================================================
## 해결 방법 : 집계 계산 처리 후 테이블화 <== 중복 제거
## =====================================================
tmp = df.groupby(['날짜','지점'], as_index=False)['매출'].sum()

wide_ok = tmp.pivot(index='날짜', columns='지점', values='매출')
display(wide_ok)

지점,부산,서울
날짜,,
2025-01,120,100
2025-02,200,90


In [ ]:
## =====================================================
## 해결방법 2 : pivot_table()
## =====================================================
## 합계로 집계
pt = pd.pivot_table(
    df,
    index='날짜',           # 행 인덱스
    columns='지점',         # 열 머리
    values='매출',          # 값
    aggfunc='sum',         # 집계 방식(기본은 mean)
    fill_value=0           # NaN 대신 0 채우기
)

## 확인
display(pt)

지점,부산,서울
날짜,,
2025-01,120,100
2025-02,200,90


In [15]:
## ------------------------------------------
## [재구성] 날짜-지점 표로, 값은 매출
## ------------------------------------------
## 합계로 집계, 합계 컬럼 추가
pt_sum = pd.pivot_table( df,
                         index='날짜',
                         columns='지점',
                         values='매출',
                         aggfunc='sum',
                         margins=True,           # 총합/평균 열·행 추가
                         margins_name='합계',     # 라벨 이름
                         fill_value=0 )

display(pt_sum)

지점,부산,서울,합계
날짜,,,
2025-01,120,100,220
2025-02,200,90,290
합계,320,190,510


In [21]:
## ------------------------------------------
## [재구성] 날짜-지점 표로, 값은 매출
## ------------------------------------------
pt_multi = pd.pivot_table(
    df,
    index=['날짜'],
    columns=['지점'],
    values=['매출','수량'],
    aggfunc={'매출':['sum','mean'], '수량':['sum','mean']}, 
    fill_value=0
)

display(pt_multi)

매출                     수량              
          mean         sum       mean       sum    
지점          부산     서울   부산   서울    부산    서울  부산  서울
날짜                                                 
2025-01  120.0  100.0  120  100  12.0  10.0  12  10
2025-02  100.0   90.0  200   90  10.0   9.0  20   9

In [ ]:
## 멀티인덱스 확인
print(f'columns => {pt_multi.columns}')
print(f'nlevels => {pt_multi.columns.nlevels}개')

## 인덱스 구조 확인 후, 맞게 합계 행 추가
if isinstance(pt_multi.index, pd.MultiIndex):
    sum_index = tuple(['합계'] * pt_multi.index.nlevels)
else:
    sum_index = '합계'

## 행 주가
pt_multi.loc[sum_index] = pt_multi.sum(numeric_only=True)

## 컬럼 수준 수 맞게 합계 열 추가
n_levels = pt_multi.columns.nlevels
sum_col_name = tuple(['합계'] * n_levels)
pt_multi[sum_col_name] = pt_multi.sum(axis=1, numeric_only=True)

## 확인인
display(pt_multi)

columns => MultiIndex([('매출', 'mean', '부산'),
            ('매출', 'mean', '서울'),
            ('매출',  'sum', '부산'),
            ('매출',  'sum', '서울'),
            ('수량', 'mean', '부산'),
            ('수량', 'mean', '서울'),
            ('수량',  'sum', '부산'),
            ('수량',  'sum', '서울'),
            ('합계',   '합계', '합계')],
           names=[None, None, '지점'])
nlevels => 3개


매출                         수량                        합계
          mean           sum         mean         sum            합계
지점          부산     서울     부산     서울    부산    서울    부산    서울      합계
날짜                                                                 
2025-01  120.0  100.0  120.0  100.0  12.0  10.0  12.0  10.0   968.0
2025-02  100.0   90.0  200.0   90.0  10.0   9.0  20.0   9.0  1056.0
합계       220.0  190.0  320.0  190.0  22.0  19.0  32.0  19.0  2024.0